목적 : 비상장 외감기업 탐색적 데이터 분석 EDA
=====================

목표 : 기업수명주기별, 부실별, 기초통계량, 정규성, Q-Qplot
-------------------------    

import modules

In [38]:
import pandas as pd
import numpy as np
from collections import Counter

%matplotlib inline
import matplotlib.pyplot as plt 
import seaborn as sns
from scipy import stats
import statsmodels.api as sm

import platform
if platform.system() == "Darwin":           # MAC인 경우
    plt.rc('font', family ="AppleGothic")
else:
    plt.rc('font',family='Malgun Gothic')

#멧플로립에서 음수를 표시
plt.rc('axes',unicode_minus=False)

#워닝 메시지
import warnings
warnings.filterwarnings('ignore')


import scipy.stats as stats
from scipy.stats import shapiro, pearsonr
from itertools import combinations
from sklearn.model_selection import train_test_split

환경설정

In [39]:
# pd.set_option('display.max_rows', None)
pd.reset_option('display.max_rows')

# pd.set_option('display.max_columns', None)
pd.reset_option('display.max_columns')

pd.set_option('display.max_seq_items', None)

pd.set_option('float_format', '{:.3f}'.format) # 소수점 3개까지 출력

load file

In [40]:
unlisted = pd.read_csv("../data/datasets/비상장수정필요피처.csv")
# 영업년수와 상장년수가 있더라

In [41]:
unlisted.columns

Index(['회사명', '거래소코드', '회계년도', 'CASH FLOW 대 부채비율', 'CASH FLOW 대 차입금비율',
       'CASH FLOW 대 총자본비율', 'CASH FLOW 대 매출액비율', '유동비율', '당좌비율', '부채비율',
       '유동부채비율', '차입금의존도', '순운전자본비율', '현금비율', '유동자산구성비율', '자기자본구성비율',
       '자기자본순이익률', '경영자본순이익률', '매출액순이익률', '총자본사업이익률', '총자본영업이익률', '매출액영업이익률',
       '금융비용부담률', 'CROE', '총자산증가율', '총자본증가율', '유동자산증가율', '비유동자산증가율', '매출액증가율',
       '유형자산증가율', '부가가치(백만원)', '총자본투자효율', '설비투자효율', '부가가치율', '이윤분배율', '자본분배율',
       '총자산회전율', '매출채권회전률', '당좌자산회전률', '유형자산회전율', '경영자본회전률', '총자본회전률',
       '비유동자산회전률', '유동자산회전률', '영업년수', '쭈피처', '영업레버리지', '재무레버리지', '영업투자현금비율',
       '영업재무현금비율', '도입기', '성숙기', '성장기', '쇠퇴기', '부실판단'],
      dtype='object')

In [42]:
unlisted

,회사명,거래소코드,회계년도,CASH FLOW 대 부채비율,CASH FLOW 대 차입금비율,CASH FLOW 대 총자본비율,CASH FLOW 대 매출액비율,유동비율,당좌비율,부채비율,...,쭈피처,영업레버리지,재무레버리지,영업투자현금비율,영업재무현금비율,도입기,성숙기,성장기,쇠퇴기,부실판단
0,주식회사성환,81567,2019,7.090,17.070,0.560,13.700,30.420,30.420,8.610,...,0.955,22.312,0.423,-1.365,-3.611,0,1,0,0,0.000
1,주식회사성후,86355,2022,8.060,22.990,5.920,80.120,211.530,211.530,276.770,...,0.319,1.468,1.000,-0.218,0.287,0,0,1,0,0.000
2,주식회사성훈,94984,2021,-0.660,-0.690,-0.680,-13.770,9.300,9.230,0.000,...,1.134,-2.671,0.308,8.616,-0.814,1,0,0,0,1.000
3,주식회사성훈철강,56923,2022,23.220,30.130,7.230,12.350,65.240,50.340,45.270,...,0.060,1.383,0.849,-1.465,-10.146,0,1,0,0,0.000
4,주식회사성희,63893,2021,-11.770,-12.620,-8.050,-1202.080,141.010,3.010,216.010,...,11.439,-0.096,2.651,-2.110,-5.661,0,0,0,1,1.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20702,주식회사나노브릭,77738,2017,-14.230,-27.590,-13.900,-41.830,20.670,9.690,4278.150,...,0.395,-0.227,0.470,-0.235,0.141,0,0,0,1,1.000
20703,주식회사나노아,67623,2017,-153.850,0.000,-49.380,-934.360,313.180,313.180,47.270,...,17.432,-0.061,-0.819,-0.586,1.368,0,0,0,1,0.000
20704,주식회사나노앤텍,72817,2022,0.760,0.850,0.570,1.400,63.520,34.190,304.840,...,0.091,3.475,29.484,-0.015,0.016,0,0,1,0,0.000
20705,주식회사나노코,56825,2022,28.060,38.700,11.230,11.220,261.670,184.980,66.720,...,0.082,1.889,0.954,-1.021,-10.925,0,1,0,0,0.000


# train test split

In [43]:
X = unlisted.drop('부실판단', axis=1)
y = unlisted['부실판단']
# 데이터를 학습용과 테스트용으로 나눔
unlisted_X_train, unlisted_X_test, unlisted_y_train, unlisted_y_test = train_test_split(X, y, test_size= 1 / (np.exp(1) + 1) , random_state=911, stratify=y)

In [44]:
unlisted_X_train['부실판단'] = unlisted_y_train

In [45]:
len(X)

20707

In [46]:
len(unlisted_X_train)

15138

In [47]:
len(unlisted_X_test)

5569

# describe

all

In [48]:
unlisted_X_train

,회사명,거래소코드,회계년도,CASH FLOW 대 부채비율,CASH FLOW 대 차입금비율,CASH FLOW 대 총자본비율,CASH FLOW 대 매출액비율,유동비율,당좌비율,부채비율,...,쭈피처,영업레버리지,재무레버리지,영업투자현금비율,영업재무현금비율,도입기,성숙기,성장기,쇠퇴기,부실판단
7716,(주)동남코리아,70118,2022,-18.740,-39.580,-3.410,-27.330,26.090,17.940,22.200,...,0.414,-1.063,-0.394,76.854,-1.323,1,0,0,0,0.000
4739,피에스케이홀딩스(주),20666,2019,130.010,22131.400,10.600,36.690,1385.060,1230.540,8.880,...,0.289,2.414,0.503,-1.195,-3.927,0,1,0,0,0.000
12685,금용해양산업주식회사,63939,2022,121.510,368.960,29.460,46.110,159.900,159.900,32.010,...,0.049,1.080,0.999,-2.679,-1.741,0,1,0,0,0.000
17405,주식회사대일,53349,2014,-2.390,-3.210,-1.850,-2.010,87.410,23.480,341.770,...,0.102,2.030,1.585,0.070,-0.142,1,0,0,0,1.000
18617,주식회사삼호프레스,83444,2018,-1.480,-2.030,-0.910,-2.540,71.900,59.910,160.760,...,0.237,6.115,0.447,-0.385,0.462,0,0,0,1,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13013,농업회사법인한만두식품주식회사,82850,2022,6.720,8.850,5.280,5.950,95.860,63.560,367.770,...,0.172,4.615,1.414,-0.155,0.420,0,0,1,0,0.000
12657,금성냉장주식회사,29681,2014,-0.530,-1.020,-0.440,-0.520,28.600,13.240,484.900,...,0.083,-0.938,0.937,-1.821,-2.265,0,1,0,0,1.000
11925,(주)노비아레텍,70975,2022,-7.670,-11.400,-5.630,-4.340,100.960,38.140,276.910,...,0.069,7.126,1.497,3.183,-0.746,1,0,0,0,1.000
10995,새한에프앤비(주),13761,2014,-9.190,-10.580,-6.360,-17.210,45.060,45.060,224.730,...,1.143,-7.013,1.058,-0.725,2.522,0,0,0,1,1.000


In [49]:
unlisted_X_train.describe()

,거래소코드,회계년도,CASH FLOW 대 부채비율,CASH FLOW 대 차입금비율,CASH FLOW 대 총자본비율,CASH FLOW 대 매출액비율,유동비율,당좌비율,부채비율,유동부채비율,...,쭈피처,영업레버리지,재무레버리지,영업투자현금비율,영업재무현금비율,도입기,성숙기,성장기,쇠퇴기,부실판단
count,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,...,15138.000,15138.000,15137.000,15138.000,14908.000,15138.000,15138.000,15138.000,15138.000,15138.000
mean,67773.011,2019.874,19.314,724.234,3.241,-22.741,431.245,358.475,1378.936,983.416,...,0.503,9.588,0.895,117.567,-5.947,0.218,0.368,0.191,0.223,0.361
std,23619.347,2.797,1261.272,31970.320,37.792,539.239,5347.663,5224.565,35986.322,30885.126,...,2.602,364.674,32.023,30082.690,678.697,0.413,0.482,0.393,0.416,0.480
min,10037.000,2014.000,-86467.650,-323703.660,-3388.620,-21110.050,0.070,0.070,0.000,0.000,...,0.000,-9823.523,-3383.817,-1368867.400,-35585.135,0.000,0.000,0.000,0.000,0.000
25%,52329.000,2018.000,-4.480,-5.120,-2.880,-4.890,56.050,31.620,60.845,36.860,...,0.086,-0.136,0.453,-2.527,-1.760,0.000,0.000,0.000,0.000,0.000
50%,70853.000,2022.000,2.920,1.510,1.860,2.360,108.715,72.700,163.855,102.565,...,0.162,2.100,0.956,-0.802,-0.770,0.000,0.000,0.000,0.000,0.000
75%,87046.750,2022.000,14.328,17.380,7.387,10.777,197.823,150.323,373.438,239.108,...,0.393,4.274,1.352,-0.063,0.216,0.000,1.000,0.000,0.000,1.000
max,101635.000,2022.000,113999.840,3194164.340,1274.750,21579.540,447760.110,447760.110,3385006.150,2905990.690,...,183.355,31473.399,743.445,3001862.889,32320.540,1.000,1.000,1.000,1.000,1.000


In [50]:
unlisted_X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15138 entries, 7716 to 12115
Data columns (total 55 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   회사명                15138 non-null  object 
 1   거래소코드              15138 non-null  int64  
 2   회계년도               15138 non-null  int64  
 3   CASH FLOW 대 부채비율   15138 non-null  float64
 4   CASH FLOW 대 차입금비율  15138 non-null  float64
 5   CASH FLOW 대 총자본비율  15138 non-null  float64
 6   CASH FLOW 대 매출액비율  15138 non-null  float64
 7   유동비율               15138 non-null  float64
 8   당좌비율               15138 non-null  float64
 9   부채비율               15138 non-null  float64
 10  유동부채비율             15138 non-null  float64
 11  차입금의존도             15138 non-null  float64
 12  순운전자본비율            15138 non-null  float64
 13  현금비율               15138 non-null  float64
 14  유동자산구성비율           15138 non-null  float64
 15  자기자본구성비율           15138 non-null  float64
 16  자기자본순이익률           15138

In [51]:
unlisted_X_train.columns

Index(['회사명', '거래소코드', '회계년도', 'CASH FLOW 대 부채비율', 'CASH FLOW 대 차입금비율',
       'CASH FLOW 대 총자본비율', 'CASH FLOW 대 매출액비율', '유동비율', '당좌비율', '부채비율',
       '유동부채비율', '차입금의존도', '순운전자본비율', '현금비율', '유동자산구성비율', '자기자본구성비율',
       '자기자본순이익률', '경영자본순이익률', '매출액순이익률', '총자본사업이익률', '총자본영업이익률', '매출액영업이익률',
       '금융비용부담률', 'CROE', '총자산증가율', '총자본증가율', '유동자산증가율', '비유동자산증가율', '매출액증가율',
       '유형자산증가율', '부가가치(백만원)', '총자본투자효율', '설비투자효율', '부가가치율', '이윤분배율', '자본분배율',
       '총자산회전율', '매출채권회전률', '당좌자산회전률', '유형자산회전율', '경영자본회전률', '총자본회전률',
       '비유동자산회전률', '유동자산회전률', '영업년수', '쭈피처', '영업레버리지', '재무레버리지', '영업투자현금비율',
       '영업재무현금비율', '도입기', '성숙기', '성장기', '쇠퇴기', '부실판단'],
      dtype='object')

In [52]:
unlisted_X_train.isna().sum().sort_values(ascending=False)

영업재무현금비율             230
재무레버리지                 1
회사명                    0
경영자본회전률                0
부가가치(백만원)              0
총자본투자효율                0
설비투자효율                 0
부가가치율                  0
이윤분배율                  0
자본분배율                  0
총자산회전율                 0
매출채권회전률                0
당좌자산회전률                0
유형자산회전율                0
비유동자산회전률               0
총자본회전률                 0
매출액증가율                 0
유동자산회전률                0
영업년수                   0
쭈피처                    0
영업레버리지                 0
영업투자현금비율               0
도입기                    0
성숙기                    0
성장기                    0
쇠퇴기                    0
유형자산증가율                0
비유동자산증가율               0
거래소코드                  0
현금비율                   0
회계년도                   0
CASH FLOW 대 부채비율       0
CASH FLOW 대 차입금비율      0
CASH FLOW 대 총자본비율      0
CASH FLOW 대 매출액비율      0
유동비율                   0
당좌비율                   0
부채비율                   0
유동부채비율                 0
차입금의존도                 0


In [53]:
unlisted_X_train.drop(['부실판단', '도입기', '성장기', '성숙기', '쇠퇴기'], axis=1).isin([0]).sum().sort_values(ascending=False) / len(unlisted_X_train) * 100

자본분배율               10.556
이윤분배율                9.169
총자산회전율               8.944
총자산증가율               8.938
차입금의존도               8.409
CASH FLOW 대 차입금비율    8.376
매출채권회전률              7.802
자기자본순이익률             4.987
자기자본구성비율             3.970
부채비율                 3.957
유동부채비율               3.957
유형자산증가율              2.299
설비투자효율               1.427
유형자산회전율              1.249
매출액증가율               0.958
현금비율                 0.595
금융비용부담률              0.548
총자본회전률               0.357
비유동자산증가율             0.304
경영자본회전률              0.244
비유동자산회전률             0.185
영업년수                 0.106
유동자산회전률              0.092
쭈피처                  0.086
총자본사업이익률             0.059
총자본증가율               0.053
총자본영업이익률             0.053
매출액영업이익률             0.053
CASH FLOW 대 총자본비율    0.053
CASH FLOW 대 매출액비율    0.053
경영자본순이익률             0.046
CASH FLOW 대 부채비율     0.033
유동자산증가율              0.026
매출액순이익률              0.026
총자본투자효율              0.026
당좌자산회전률              0.013
CROE                 0.013
부

In [54]:
unlisted_X_train.isin([np.inf]).sum().sort_values(ascending=False)

총자산증가율               1
회사명                  0
매출액증가율               0
부가가치(백만원)            0
총자본투자효율              0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
총자본회전률               0
비유동자산회전률             0
유동자산회전률              0
영업년수                 0
쭈피처                  0
영업레버리지               0
재무레버리지               0
영업투자현금비율             0
영업재무현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
유형자산증가율              0
비유동자산증가율             0
거래소코드                0
유동자산증가율              0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율              0
현금비율                 0
유동자산구성비율   

In [55]:
unlisted_X_train.isin([-np.inf]).sum().sort_values(ascending=False)

회사명                  0
총자본회전률               0
부가가치(백만원)            0
총자본투자효율              0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
비유동자산회전률             0
매출액증가율               0
유동자산회전률              0
영업년수                 0
쭈피처                  0
영업레버리지               0
재무레버리지               0
영업투자현금비율             0
영업재무현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
유형자산증가율              0
비유동자산증가율             0
거래소코드                0
현금비율                 0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율              0
유동자산구성비율             0
유동자산증가율              0
자기자본구성비율   

In [56]:
# inf_col = unlisted_X_train.isin([np.inf]).sum().sort_values(ascending=False).index[0]
# max_value = unlisted_X_train[inf_col].replace(np.inf, np.nan).max()
# unlisted_X_train[inf_col] = unlisted_X_train[inf_col].replace(np.inf, max_value)

# 범주 분할 1현금흐름 / 2안정성 / 2수익성 / 3유동성 / 4성장성 / 5생산성 / 6활동성 / 7시장지표 및 기타

In [57]:
# 비상장 all data는 unlisted
# 비상장 label들

labels = unlisted_X_train[
    [
        '거래소코드',
        '회계년도',
        '부실판단',
        '도입기',
        '성숙기',
        '성장기',
        '쇠퇴기'
    ]
]

# 비상장 피처
features = unlisted_X_train.drop(
    labels=[
        '회사명',
        '거래소코드',
        '회계년도',        
        '도입기',
        '성숙기',
        '성장기',
        '쇠퇴기',
        '부실판단'        
    ],
    axis=1,    
)

# 1현금흐름 4개
cash_ft1 = unlisted_X_train[
    [
        'CASH FLOW 대 부채비율',
        'CASH FLOW 대 차입금비율',
        'CASH FLOW 대 총자본비율',
        'CASH FLOW 대 매출액비율',
        '부실판단'
    ]
]

# 2안정성 9개
stability_ft1 = unlisted_X_train[
    [
        '유동비율',
        '당좌비율',
        '부채비율',
        '유동부채비율',
        '차입금의존도',
        '순운전자본비율',
        '현금비율',
        '유동자산구성비율',
        '자기자본구성비율',
        '부실판단'
    ]
]

# 3수익성 8개
profitability_ft1 = unlisted_X_train[
    [
        '자기자본순이익률',
        '경영자본순이익률',
        '매출액순이익률',
        '총자본사업이익률',
        '총자본영업이익률',
        '매출액영업이익률',
        '금융비용부담률',
        'CROE',
        '부실판단'
    ]
]

# 4성장성 6개
growth_ft1 = unlisted_X_train[
    [
        '총자산증가율',
        '총자본증가율',
        '유동자산증가율',
        '비유동자산증가율',
        '매출액증가율',
        '유형자산증가율',
        '부실판단'
    ]
]

# 5생산성 6개
productivity_ft1 = unlisted_X_train[
    [
        '부가가치(백만원)',
        '총자본투자효율',
        '설비투자효율',
        '부가가치율',
        '이윤분배율',
        '자본분배율',
        '부실판단'
    ]
]

# 6활동성 8개
activity_ft1 = unlisted_X_train[
    [
        '총자산회전율',
        '매출채권회전률',
        '당좌자산회전률',
        '유형자산회전율',
        '경영자본회전률',
        '총자본회전률',
        '비유동자산회전률',
        '유동자산회전률',
        '부실판단'
    ]
]

# 7기타
etc_ft1 = unlisted_X_train[
    [
        '영업년수',
        '쭈피처',
        '영업레버리지',
        '재무레버리지',
        '영업투자현금비율',
        '영업재무현금비율', 
        '도입기',
        '성숙기',
        '성장기',
        '쇠퇴기',
        '부실판단'
    ]
]

In [58]:
unlisted_X_train.columns

Index(['회사명', '거래소코드', '회계년도', 'CASH FLOW 대 부채비율', 'CASH FLOW 대 차입금비율',
       'CASH FLOW 대 총자본비율', 'CASH FLOW 대 매출액비율', '유동비율', '당좌비율', '부채비율',
       '유동부채비율', '차입금의존도', '순운전자본비율', '현금비율', '유동자산구성비율', '자기자본구성비율',
       '자기자본순이익률', '경영자본순이익률', '매출액순이익률', '총자본사업이익률', '총자본영업이익률', '매출액영업이익률',
       '금융비용부담률', 'CROE', '총자산증가율', '총자본증가율', '유동자산증가율', '비유동자산증가율', '매출액증가율',
       '유형자산증가율', '부가가치(백만원)', '총자본투자효율', '설비투자효율', '부가가치율', '이윤분배율', '자본분배율',
       '총자산회전율', '매출채권회전률', '당좌자산회전률', '유형자산회전율', '경영자본회전률', '총자본회전률',
       '비유동자산회전률', '유동자산회전률', '영업년수', '쭈피처', '영업레버리지', '재무레버리지', '영업투자현금비율',
       '영업재무현금비율', '도입기', '성숙기', '성장기', '쇠퇴기', '부실판단'],
      dtype='object')

1현금흐름 4가지 cash

In [59]:
cash_ft1.describe()

,CASH FLOW 대 부채비율,CASH FLOW 대 차입금비율,CASH FLOW 대 총자본비율,CASH FLOW 대 매출액비율,부실판단
count,15138.000,15138.000,15138.000,15138.000,15138.000
mean,19.314,724.234,3.241,-22.741,0.361
std,1261.272,31970.320,37.792,539.239,0.480
min,-86467.650,-323703.660,-3388.620,-21110.050,0.000
25%,-4.480,-5.120,-2.880,-4.890,0.000
50%,2.920,1.510,1.860,2.360,0.000
75%,14.328,17.380,7.387,10.777,1.000
max,113999.840,3194164.340,1274.750,21579.540,1.000


2안정성 9가지 stability

In [60]:
stability_ft1.describe()

,유동비율,당좌비율,부채비율,유동부채비율,차입금의존도,순운전자본비율,현금비율,유동자산구성비율,자기자본구성비율,부실판단
count,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000
mean,431.245,358.475,1378.936,983.416,43.176,0.167,98.637,45.603,38.894,0.361
std,5347.663,5224.565,35986.322,30885.126,211.937,297.974,2442.606,28.781,26.118,0.480
min,0.070,0.070,0.000,0.000,0.000,-36261.200,-0.200,0.000,0.000,0.000
25%,56.050,31.620,60.845,36.860,17.360,-19.517,1.590,21.870,18.270,0.000
50%,108.715,72.700,163.855,102.565,42.040,2.925,8.605,43.180,35.145,0.000
75%,197.823,150.323,373.438,239.108,61.450,26.938,32.528,67.630,57.267,1.000
max,447760.110,447760.110,3385006.150,2905990.690,25856.360,99.690,275285.960,100.000,99.980,1.000


3수익성 8가지 profitability

In [61]:
productivity_ft1.describe()

,부가가치(백만원),총자본투자효율,설비투자효율,부가가치율,이윤분배율,자본분배율,부실판단
count,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000
mean,6674.974,30.528,538176.590,46.170,-95.676,8.411,0.361
std,352176.022,1669.910,34787344.705,747.415,2660.033,1074.584,0.480
min,-68167.780,-246.020,-563419210.000,-13763.270,-208392.960,-94627.640,0.000
25%,937.122,4.870,9.130,7.960,-0.310,22.102,0.000
50%,2281.575,11.060,28.820,16.220,11.390,55.870,0.000
75%,4883.170,19.218,97.688,33.188,35.943,74.770,1.000
max,43327598.520,205392.780,4053778655.000,70947.820,8226.360,126.190,1.000


4성장성 6개 growth

In [62]:
growth_ft1.describe()

,총자산증가율,총자본증가율,유동자산증가율,비유동자산증가율,매출액증가율,유형자산증가율,부실판단
count,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000
mean,inf,6.597,216.233,302.142,228.063,192381.296,0.361
std,NaN,55.243,20598.824,11247.279,9908.536,21029765.042,0.480
min,-100.000,-99.890,-99.900,-100.000,-99.910,-100.000,0.000
25%,-4.447,-5.480,-14.060,-5.207,-12.510,-5.630,0.000
50%,0.523,1.820,3.990,-0.110,3.610,-1.040,0.000
75%,11.191,12.805,25.038,12.480,21.797,8.002,1.000
max,inf,5081.220,2531488.340,1154781.210,900275.000,2580453089.000,1.000


5생산성 6가지 productivity

In [63]:
productivity_ft1.describe()

,부가가치(백만원),총자본투자효율,설비투자효율,부가가치율,이윤분배율,자본분배율,부실판단
count,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000
mean,6674.974,30.528,538176.590,46.170,-95.676,8.411,0.361
std,352176.022,1669.910,34787344.705,747.415,2660.033,1074.584,0.480
min,-68167.780,-246.020,-563419210.000,-13763.270,-208392.960,-94627.640,0.000
25%,937.122,4.870,9.130,7.960,-0.310,22.102,0.000
50%,2281.575,11.060,28.820,16.220,11.390,55.870,0.000
75%,4883.170,19.218,97.688,33.188,35.943,74.770,1.000
max,43327598.520,205392.780,4053778655.000,70947.820,8226.360,126.190,1.000


6활동성 8가지 activity

In [64]:
activity_ft1.describe()

,총자산회전율,매출채권회전률,당좌자산회전률,유형자산회전율,경영자본회전률,총자본회전률,비유동자산회전률,유동자산회전률,부실판단
count,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000
mean,0.938,700.636,4.246,7686.666,1.119,0.996,22.988,2.830,0.361
std,1.663,46090.572,7.518,263256.703,1.590,1.404,375.217,5.156,0.480
min,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,0.162,3.990,1.460,0.650,0.340,0.280,0.490,1.060,0.000
50%,0.652,7.150,2.880,2.060,0.820,0.730,1.460,1.990,0.000
75%,1.231,13.370,4.940,7.050,1.430,1.300,3.757,3.340,1.000
max,100.540,5386048.240,344.060,19823298.290,106.800,92.050,32610.280,344.060,1.000


7기타 etc

In [65]:
etc_ft1.describe()

,영업년수,쭈피처,영업레버리지,재무레버리지,영업투자현금비율,영업재무현금비율,도입기,성숙기,성장기,쇠퇴기,부실판단
count,15138.000,15138.000,15138.000,15137.000,15138.000,14908.000,15138.000,15138.000,15138.000,15138.000,15138.000
mean,21.322,0.503,9.588,0.895,117.567,-5.947,0.218,0.368,0.191,0.223,0.361
std,11.151,2.602,364.674,32.023,30082.690,678.697,0.413,0.482,0.393,0.416,0.480
min,0.000,0.000,-9823.523,-3383.817,-1368867.400,-35585.135,0.000,0.000,0.000,0.000,0.000
25%,13.000,0.086,-0.136,0.453,-2.527,-1.760,0.000,0.000,0.000,0.000,0.000
50%,20.000,0.162,2.100,0.956,-0.802,-0.770,0.000,0.000,0.000,0.000,0.000
75%,27.000,0.393,4.274,1.352,-0.063,0.216,0.000,1.000,0.000,0.000,1.000
max,110.000,183.355,31473.399,743.445,3001862.889,32320.540,1.000,1.000,1.000,1.000,1.000


# 도입정상

In [66]:
도입정상 = unlisted_X_train[(unlisted_X_train['도입기'] == 1) & (unlisted_X_train['부실판단'] == 0)]

도입정상 결측치확인

In [67]:
도입정상.isna().sum().sort_values(ascending=False)

회사명                  0
총자본회전률               0
부가가치(백만원)            0
총자본투자효율              0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
비유동자산회전률             0
매출액증가율               0
유동자산회전률              0
영업년수                 0
쭈피처                  0
영업레버리지               0
재무레버리지               0
영업투자현금비율             0
영업재무현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
유형자산증가율              0
비유동자산증가율             0
거래소코드                0
현금비율                 0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율              0
유동자산구성비율             0
유동자산증가율              0
자기자본구성비율   

# 도입부실

In [68]:
도입부실 = unlisted_X_train[(unlisted_X_train['도입기'] == 1) & (unlisted_X_train['부실판단'] == 1)]

도입부실 결측치확인

In [69]:
도입부실.isna().sum().sort_values(ascending=False)

회사명                  0
총자본회전률               0
부가가치(백만원)            0
총자본투자효율              0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
비유동자산회전률             0
매출액증가율               0
유동자산회전률              0
영업년수                 0
쭈피처                  0
영업레버리지               0
재무레버리지               0
영업투자현금비율             0
영업재무현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
유형자산증가율              0
비유동자산증가율             0
거래소코드                0
현금비율                 0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율              0
유동자산구성비율             0
유동자산증가율              0
자기자본구성비율   

정규성 확인

In [32]:
# na_col = 도입부실.isna().sum().sort_values(ascending=False).index[0]

In [33]:
# 테스트를 위한 샘플 데이터 생성
# data = 도입부실[na_col].dropna()

In [35]:
# from scipy.stats import shapiro

# # 샤피로-윌크 검정 수행
# statistic, p_value = shapiro(data)

# # 검정 통계량과 p-value 출력
# print(f"검정 통계량: {statistic}, p-value: {p_value}")

# # p-value를 기준으로 귀무가설을 기각 여부 판단
# alpha = 0.05
# if p_value > alpha:
#     print("데이터는 정규 분포를 따릅니다 (귀무가설을 기각하지 않음)")
# else:
#     print("데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)")


TypeError: ufunc 'divide' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [36]:
# from scipy.stats import anderson

# # 안더슨-달링 검정 수행
# result = anderson(data)

# # 검정 통계량과 임계값 출력
# print(f"검정 통계량: {result.statistic}")

# # 임계값과 유의수준에 따른 결과 출력
# for i in range(len(result.critical_values)):
#     sl, cv = result.significance_level[i], result.critical_values[i]
#     print(f"유의수준 {sl}%: 임계값 = {cv}")

# # 검정 통계량이 임계값보다 작으면 정규 분포를 따른다고 판단
# if result.statistic < result.critical_values[2]:
#     print("데이터는 정규 분포를 따릅니다.")
# else:
#     print("데이터는 정규 분포를 따르지 않습니다.")


TypeError: Could not convert string '주식회사대일(주)금송(주)도로오토모티브신평산업주식회사주식회사학산플러스(주)포랑토이푸딩주식회사주식회사인맥시리얼마이크로일렉트로닉스코리아(주)(주)씨네일레븐(주)보아스에셋(주)동진앤컴퍼니태경에스비씨주식회사주식회사티쓰리엔터테인먼트(주)셀코스에스원텍주식회사농업회사법인대원바이오(주)지희산업주식회사풀무원건강생활(주)신화련금수산장개발주식회사(주)보람씨앤에치주식회사라메디텍신세기산업(주)대홍전기(주)(주)월드뷰건설주식회사센트럴시티신일산업주식회사주식회사성훈주식회사에이치앤(주)쌍용선재주식회사웰메이드생활건강주식회사프라임타임동신개발(주)주식회사벽강삼진철강주식회사주식회사모던필라테스예체능지성이노베이션(주)주식회사알멕주식회사오성기공주식회사바이오데이즈(주)서진인터내셔날이영산업기계주식회사(주)두리콜렉션주식회사유진테크놀로지노블랜드주식회사대성특수강관(주)(주)디에스플러스텍주식회사티스템(주)비에스건설하우징주식회사지에스티코리아주식회사정풍개발디에이치통상주식회사우리시스템(주)제일엠앤에스(주)희종종합건설주식회사에스쁘아동오정밀주식회사주식회사삼우코리아샬롬산업주식회사금오이엠에스(주)동대문호텔피에프브이주식회사제이더블유메디칼주식회사주식회사선용어패럴주식회사원일유통(주)우진메탈주식회사청담이앤씨제이더블유엠주식회사주식회사풀린키(주)삼성잉크삼호철강(주)주식회사창대디앤씨서진공조(주)(주)메디파마플랜서광기연(주)(주)케이디글로벌주식회사중앙종합건설주식회사스파밸리한흥물산(주)(주)케이티앤씨파워풀글로벌코리아주식회사지우산업개발(주)주식회사한국필름(주)고려주택건설(주)블랭크코퍼레이션주식회사엠케이홀딩스삼우산기주식회사(주)메르코(주)부민보드주식회사시지트로닉스(주)디디에스성운개발(주)삶도시주식회사주식회사튤립코리아주식회사스푼라디오주식회사경성리츠(주)메가테크(주)알케이에프앤리즈건설(주)주식회사정화포장주식회사가양(주)미건넥스콘테크놀러지주식회사대라주택건설(주)(주)켐옵틱스주식회사지앤브이코스메틱주식회사범아(주)펩트론(주)명실상부(주)셰프라인주식회사파인웰동국상선주식회사호산이엔지(주)주식

In [ ]:
# from scipy.stats import kstest, norm

# # Kolmogorov-Smirnov 검정 수행
# statistic, p_value = kstest(data, 'norm')

# # 검정 통계량과 p-value 출력
# print(f"검정 통계량: {statistic}, p-value: {p_value}")

# # p-value를 기준으로 귀무가설을 기각 여부 판단
# alpha = 0.05
# if p_value > alpha:
#     print("데이터는 정규 분포를 따릅니다 (귀무가설을 기각하지 않음)")
# else:
#     print("데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)")


검정 통계량: 0.5023366544630479, p-value: 0.0
데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)


fillna median

In [ ]:
# 도입부실[na_col].fillna(data.median(), inplace=True)

# 성장정상

In [74]:
성장정상 = unlisted_X_train[(unlisted_X_train['성장기'] == 1) & (unlisted_X_train['부실판단'] == 0)]

결측치확인

In [75]:
성장정상.columns

Index(['회사명', '거래소코드', '회계년도', 'CASH FLOW 대 부채비율', 'CASH FLOW 대 차입금비율',
       'CASH FLOW 대 총자본비율', 'CASH FLOW 대 매출액비율', '유동비율', '당좌비율', '부채비율',
       '유동부채비율', '차입금의존도', '순운전자본비율', '현금비율', '유동자산구성비율', '자기자본구성비율',
       '자기자본순이익률', '경영자본순이익률', '매출액순이익률', '총자본사업이익률', '총자본영업이익률', '매출액영업이익률',
       '금융비용부담률', 'CROE', '총자산증가율', '총자본증가율', '유동자산증가율', '비유동자산증가율', '매출액증가율',
       '유형자산증가율', '부가가치(백만원)', '총자본투자효율', '설비투자효율', '부가가치율', '이윤분배율', '자본분배율',
       '총자산회전율', '매출채권회전률', '당좌자산회전률', '유형자산회전율', '경영자본회전률', '총자본회전률',
       '비유동자산회전률', '유동자산회전률', '영업년수', '쭈피처', '영업레버리지', '재무레버리지', '영업투자현금비율',
       '영업재무현금비율', '도입기', '성숙기', '성장기', '쇠퇴기', '부실판단'],
      dtype='object')

In [76]:
성장정상.isna().sum().sort_values(ascending=False)

회사명                  0
총자본회전률               0
부가가치(백만원)            0
총자본투자효율              0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
비유동자산회전률             0
매출액증가율               0
유동자산회전률              0
영업년수                 0
쭈피처                  0
영업레버리지               0
재무레버리지               0
영업투자현금비율             0
영업재무현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
유형자산증가율              0
비유동자산증가율             0
거래소코드                0
현금비율                 0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율              0
유동자산구성비율             0
유동자산증가율              0
자기자본구성비율   

정규성 확인

In [ ]:
# na_col = 성장정상.isna().sum().sort_values(ascending=False).index[0]
# max_value = unlisted_X_train[inf_col].replace(np.inf, np.nan).max()
# unlisted_X_train[inf_col] = unlisted_X_train[inf_col].replace(np.inf, max_value)

In [ ]:
# data = 성장정상[na_col].dropna()

In [77]:
# data.isna().sum()

In [78]:
# from scipy.stats import shapiro

# # 샤피로-윌크 검정 수행
# statistic, p_value = shapiro(data)

# # 검정 통계량과 p-value 출력
# print(f"검정 통계량: {statistic}, p-value: {p_value}")

# # p-value를 기준으로 귀무가설을 기각 여부 판단
# alpha = 0.05
# if p_value > alpha:
#     print("데이터는 정규 분포를 따릅니다 (귀무가설을 기각하지 않음)")
# else:
#     print("데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)")


In [79]:
# from scipy.stats import anderson

# # 안더슨-달링 검정 수행
# result = anderson(data)

# # 검정 통계량과 임계값 출력
# print(f"검정 통계량: {result.statistic}")

# # 임계값과 유의수준에 따른 결과 출력
# for i in range(len(result.critical_values)):
#     sl, cv = result.significance_level[i], result.critical_values[i]
#     print(f"유의수준 {sl}%: 임계값 = {cv}")

# # 검정 통계량이 임계값보다 작으면 정규 분포를 따른다고 판단
# if result.statistic < result.critical_values[2]:
#     print("데이터는 정규 분포를 따릅니다.")
# else:
#     print("데이터는 정규 분포를 따르지 않습니다.")


In [80]:
# from scipy.stats import kstest, norm


# # Kolmogorov-Smirnov 검정 수행
# statistic, p_value = kstest(data, 'norm')

# # 검정 통계량과 p-value 출력
# print(f"검정 통계량: {statistic}, p-value: {p_value}")

# # p-value를 기준으로 귀무가설을 기각 여부 판단
# alpha = 0.05
# if p_value > alpha:
#     print("데이터는 정규 분포를 따릅니다 (귀무가설을 기각하지 않음)")
# else:
#     print("데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)")

fillna median

In [81]:
# 성장정상[na_col].fillna(data.median(), inplace=True)

# 성장부실

In [82]:
성장부실 = unlisted_X_train[(unlisted_X_train['성장기'] == 1) & (unlisted_X_train['부실판단'] == 1)]

결측치 확인

In [83]:
성장부실.isna().sum().sort_values(ascending=False)

회사명                  0
총자본회전률               0
부가가치(백만원)            0
총자본투자효율              0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
비유동자산회전률             0
매출액증가율               0
유동자산회전률              0
영업년수                 0
쭈피처                  0
영업레버리지               0
재무레버리지               0
영업투자현금비율             0
영업재무현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
유형자산증가율              0
비유동자산증가율             0
거래소코드                0
현금비율                 0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율              0
유동자산구성비율             0
유동자산증가율              0
자기자본구성비율   

inf 확인

In [84]:
성장부실.isin([np.inf]).sum().sort_values(ascending=False)

총자산증가율               1
회사명                  0
매출액증가율               0
부가가치(백만원)            0
총자본투자효율              0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
총자본회전률               0
비유동자산회전률             0
유동자산회전률              0
영업년수                 0
쭈피처                  0
영업레버리지               0
재무레버리지               0
영업투자현금비율             0
영업재무현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
유형자산증가율              0
비유동자산증가율             0
거래소코드                0
유동자산증가율              0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율              0
현금비율                 0
유동자산구성비율   

In [85]:
inf_col = 성장부실.isin([np.inf]).sum().sort_values(ascending=False).index[0]
max_value = 성장부실[inf_col].replace(np.inf, np.nan).max()
성장부실[inf_col] = 성장부실[inf_col].replace(np.inf, max_value)

In [86]:
성장부실.isin([np.inf]).sum().sort_values(ascending=False)

회사명                  0
총자본회전률               0
부가가치(백만원)            0
총자본투자효율              0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
비유동자산회전률             0
매출액증가율               0
유동자산회전률              0
영업년수                 0
쭈피처                  0
영업레버리지               0
재무레버리지               0
영업투자현금비율             0
영업재무현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
유형자산증가율              0
비유동자산증가율             0
거래소코드                0
현금비율                 0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율              0
유동자산구성비율             0
유동자산증가율              0
자기자본구성비율   

# 성숙정상

In [87]:
성숙정상 = unlisted_X_train[(unlisted_X_train['성숙기'] == 1) & (unlisted_X_train['부실판단'] == 0)]

성숙정상 결측치 확인

In [88]:
성숙정상.isna().sum().sort_values(ascending=False)

회사명                  0
총자본회전률               0
부가가치(백만원)            0
총자본투자효율              0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
비유동자산회전률             0
매출액증가율               0
유동자산회전률              0
영업년수                 0
쭈피처                  0
영업레버리지               0
재무레버리지               0
영업투자현금비율             0
영업재무현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
유형자산증가율              0
비유동자산증가율             0
거래소코드                0
현금비율                 0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율              0
유동자산구성비율             0
유동자산증가율              0
자기자본구성비율   

정규성 확인

In [89]:
# na_col = 성숙정상.isna().sum().sort_values(ascending=False).index[0]

In [90]:
len(성숙정상)

4660

In [91]:
# 테스트를 위한 샘플 데이터 생성
# data = 성숙정상[na_col].dropna()

In [92]:
# from scipy.stats import shapiro

# # 샤피로-윌크 검정 수행
# statistic, p_value = shapiro(data)

# # 검정 통계량과 p-value 출력
# print(f"검정 통계량: {statistic}, p-value: {p_value}")

# # p-value를 기준으로 귀무가설을 기각 여부 판단
# alpha = 0.05
# if p_value > alpha:
#     print("데이터는 정규 분포를 따릅니다 (귀무가설을 기각하지 않음)")
# else:
#     print("데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)")

In [93]:
# from scipy.stats import anderson

# # 안더슨-달링 검정 수행
# result = anderson(data)

# # 검정 통계량과 임계값 출력
# print(f"검정 통계량: {result.statistic}")

# # 임계값과 유의수준에 따른 결과 출력
# for i in range(len(result.critical_values)):
#     sl, cv = result.significance_level[i], result.critical_values[i]
#     print(f"유의수준 {sl}%: 임계값 = {cv}")

# # 검정 통계량이 임계값보다 작으면 정규 분포를 따른다고 판단
# if result.statistic < result.critical_values[2]:
#     print("데이터는 정규 분포를 따릅니다.")
# else:
#     print("데이터는 정규 분포를 따르지 않습니다.")

In [94]:
# from scipy.stats import kstest, norm

# # Kolmogorov-Smirnov 검정 수행
# statistic, p_value = kstest(data, 'norm')

# # 검정 통계량과 p-value 출력
# print(f"검정 통계량: {statistic}, p-value: {p_value}")

# # p-value를 기준으로 귀무가설을 기각 여부 판단
# alpha = 0.05
# if p_value > alpha:
#     print("데이터는 정규 분포를 따릅니다 (귀무가설을 기각하지 않음)")
# else:
#     print("데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)")

fillna median

In [95]:
# 성숙정상.fillna(data.median(), inplace=True)

# 성숙부실

In [96]:
성숙부실 = unlisted_X_train[(unlisted_X_train['성숙기'] == 1) & (unlisted_X_train['부실판단'] == 1)]

성숙부실 결측치 확인

In [98]:
성숙부실.isna().sum().sort_values(ascending=False)

재무레버리지               1
회사명                  0
총자본회전률               0
부가가치(백만원)            0
총자본투자효율              0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
비유동자산회전률             0
매출액증가율               0
유동자산회전률              0
영업년수                 0
쭈피처                  0
영업레버리지               0
영업투자현금비율             0
영업재무현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
유형자산증가율              0
비유동자산증가율             0
거래소코드                0
현금비율                 0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율              0
유동자산구성비율             0
유동자산증가율              0
자기자본구성비율   

정규성 확인

In [99]:
na_col0 = 성숙부실.isna().sum().sort_values(ascending=False).index[0]
# na_col1 = 성숙부실.isna().sum().sort_values(ascending=False).index[1]
# 테스트를 위한 샘플 데이터 생성
data0 = 성숙부실[na_col0].dropna()
# data1 = 성숙부실[na_col1].dropna()

In [100]:
from scipy.stats import shapiro

# 샤피로-윌크 검정 수행
statistic, p_value = shapiro(data0)

# 검정 통계량과 p-value 출력
print(f"검정 통계량: {statistic}, p-value: {p_value}")

# p-value를 기준으로 귀무가설을 기각 여부 판단
alpha = 0.05
if p_value > alpha:
    print("데이터는 정규 분포를 따릅니다 (귀무가설을 기각하지 않음)")
else:
    print("데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)")

검정 통계량: 0.3310622572898865, p-value: 0.0
데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)


In [101]:
# from scipy.stats import shapiro

# # 샤피로-윌크 검정 수행
# statistic, p_value = shapiro(data1)

# # 검정 통계량과 p-value 출력
# print(f"검정 통계량: {statistic}, p-value: {p_value}")

# # p-value를 기준으로 귀무가설을 기각 여부 판단
# alpha = 0.05
# if p_value > alpha:
#     print("데이터는 정규 분포를 따릅니다 (귀무가설을 기각하지 않음)")
# else:
#     print("데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)")

In [102]:
from scipy.stats import anderson

# 안더슨-달링 검정 수행
result = anderson(data0)

# 검정 통계량과 임계값 출력
print(f"검정 통계량: {result.statistic}")

# 임계값과 유의수준에 따른 결과 출력
for i in range(len(result.critical_values)):
    sl, cv = result.significance_level[i], result.critical_values[i]
    print(f"유의수준 {sl}%: 임계값 = {cv}")

# 검정 통계량이 임계값보다 작으면 정규 분포를 따른다고 판단
if result.statistic < result.critical_values[2]:
    print("데이터는 정규 분포를 따릅니다.")
else:
    print("데이터는 정규 분포를 따르지 않습니다.")

검정 통계량: 185.93136271858157
유의수준 15.0%: 임계값 = 0.573
유의수준 10.0%: 임계값 = 0.653
유의수준 5.0%: 임계값 = 0.784
유의수준 2.5%: 임계값 = 0.914
유의수준 1.0%: 임계값 = 1.087
데이터는 정규 분포를 따르지 않습니다.


In [103]:
# from scipy.stats import anderson

# # 안더슨-달링 검정 수행
# result = anderson(data1)

# # 검정 통계량과 임계값 출력
# print(f"검정 통계량: {result.statistic}")

# # 임계값과 유의수준에 따른 결과 출력
# for i in range(len(result.critical_values)):
#     sl, cv = result.significance_level[i], result.critical_values[i]
#     print(f"유의수준 {sl}%: 임계값 = {cv}")

# # 검정 통계량이 임계값보다 작으면 정규 분포를 따른다고 판단
# if result.statistic < result.critical_values[2]:
#     print("데이터는 정규 분포를 따릅니다.")
# else:
#     print("데이터는 정규 분포를 따르지 않습니다.")

In [104]:
from scipy.stats import kstest, norm

# Kolmogorov-Smirnov 검정 수행
statistic, p_value = kstest(data0, 'norm')

# 검정 통계량과 p-value 출력
print(f"검정 통계량: {statistic}, p-value: {p_value}")

# p-value를 기준으로 귀무가설을 기각 여부 판단
alpha = 0.05
if p_value > alpha:
    print("데이터는 정규 분포를 따릅니다 (귀무가설을 기각하지 않음)")
else:
    print("데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)")

검정 통계량: 0.14866872064673958, p-value: 4.8506857072186544e-18
데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)


In [105]:
# from scipy.stats import kstest, norm

# # Kolmogorov-Smirnov 검정 수행
# statistic, p_value = kstest(data1, 'norm')

# # 검정 통계량과 p-value 출력
# print(f"검정 통계량: {statistic}, p-value: {p_value}")

# # p-value를 기준으로 귀무가설을 기각 여부 판단
# alpha = 0.05
# if p_value > alpha:
#     print("데이터는 정규 분포를 따릅니다 (귀무가설을 기각하지 않음)")
# else:
#     print("데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)")

fillna median

In [106]:
성숙부실[na_col0].fillna(data0.median(), inplace=True)

In [107]:
# 성숙부실[na_col1].fillna(data1.median(), inplace=True)

# 쇠퇴정상

In [108]:
쇠퇴정상 = unlisted_X_train[(unlisted_X_train['쇠퇴기'] == 1) & (unlisted_X_train['부실판단'] == 0)]

결측치 확인

In [109]:
쇠퇴정상.isna().sum().sort_values(ascending=False)

영업재무현금비율             124
회사명                    0
총자본회전률                 0
부가가치(백만원)              0
총자본투자효율                0
설비투자효율                 0
부가가치율                  0
이윤분배율                  0
자본분배율                  0
총자산회전율                 0
매출채권회전률                0
당좌자산회전률                0
유형자산회전율                0
경영자본회전률                0
비유동자산회전률               0
매출액증가율                 0
유동자산회전률                0
영업년수                   0
쭈피처                    0
영업레버리지                 0
재무레버리지                 0
영업투자현금비율               0
도입기                    0
성숙기                    0
성장기                    0
쇠퇴기                    0
유형자산증가율                0
비유동자산증가율               0
거래소코드                  0
현금비율                   0
회계년도                   0
CASH FLOW 대 부채비율       0
CASH FLOW 대 차입금비율      0
CASH FLOW 대 총자본비율      0
CASH FLOW 대 매출액비율      0
유동비율                   0
당좌비율                   0
부채비율                   0
유동부채비율                 0
차입금의존도                 0


정규성 확인

In [110]:
na_col0 = 쇠퇴정상.isna().sum().sort_values(ascending=False).index[0]
# na_col1 = 쇠퇴정상.isna().sum().sort_values(ascending=False).index[1]
# 테스트를 위한 샘플 데이터 생성
data0 = 쇠퇴정상[na_col0].dropna()
# data1 = 쇠퇴정상[na_col1].dropna()

In [111]:
from scipy.stats import shapiro

# 샤피로-윌크 검정 수행
statistic, p_value = shapiro(data0)

# 검정 통계량과 p-value 출력
print(f"검정 통계량: {statistic}, p-value: {p_value}")

# p-value를 기준으로 귀무가설을 기각 여부 판단
alpha = 0.05
if p_value > alpha:
    print("데이터는 정규 분포를 따릅니다 (귀무가설을 기각하지 않음)")
else:
    print("데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)")

검정 통계량: 0.01838862895965576, p-value: 0.0
데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)


In [112]:
# from scipy.stats import shapiro

# # 샤피로-윌크 검정 수행
# statistic, p_value = shapiro(data1)

# # 검정 통계량과 p-value 출력
# print(f"검정 통계량: {statistic}, p-value: {p_value}")

# # p-value를 기준으로 귀무가설을 기각 여부 판단
# alpha = 0.05
# if p_value > alpha:
#     print("데이터는 정규 분포를 따릅니다 (귀무가설을 기각하지 않음)")
# else:
#     print("데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)")

In [113]:
from scipy.stats import anderson

# 안더슨-달링 검정 수행
result = anderson(data0)

# 검정 통계량과 임계값 출력
print(f"검정 통계량: {result.statistic}")

# 임계값과 유의수준에 따른 결과 출력
for i in range(len(result.critical_values)):
    sl, cv = result.significance_level[i], result.critical_values[i]
    print(f"유의수준 {sl}%: 임계값 = {cv}")

# 검정 통계량이 임계값보다 작으면 정규 분포를 따른다고 판단
if result.statistic < result.critical_values[2]:
    print("데이터는 정규 분포를 따릅니다.")
else:
    print("데이터는 정규 분포를 따르지 않습니다.")

검정 통계량: 503.70699486728176
유의수준 15.0%: 임계값 = 0.574
유의수준 10.0%: 임계값 = 0.654
유의수준 5.0%: 임계값 = 0.785
유의수준 2.5%: 임계값 = 0.915
유의수준 1.0%: 임계값 = 1.089
데이터는 정규 분포를 따르지 않습니다.


In [114]:
# from scipy.stats import anderson

# # 안더슨-달링 검정 수행
# result = anderson(data1)

# # 검정 통계량과 임계값 출력
# print(f"검정 통계량: {result.statistic}")

# # 임계값과 유의수준에 따른 결과 출력
# for i in range(len(result.critical_values)):
#     sl, cv = result.significance_level[i], result.critical_values[i]
#     print(f"유의수준 {sl}%: 임계값 = {cv}")

# # 검정 통계량이 임계값보다 작으면 정규 분포를 따른다고 판단
# if result.statistic < result.critical_values[2]:
#     print("데이터는 정규 분포를 따릅니다.")
# else:
#     print("데이터는 정규 분포를 따르지 않습니다.")

In [116]:
from scipy.stats import kstest, norm

# Kolmogorov-Smirnov 검정 수행
statistic, p_value = kstest(data0, 'norm')

# 검정 통계량과 p-value 출력
print(f"검정 통계량: {statistic}, p-value: {p_value}")

# p-value를 기준으로 귀무가설을 기각 여부 판단
alpha = 0.05
if p_value > alpha:
    print("데이터는 정규 분포를 따릅니다 (귀무가설을 기각하지 않음)")
else:

    print("데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)")

검정 통계량: 0.1679789483977252, p-value: 6.862637710098656e-34
데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)


In [117]:
# from scipy.stats import kstest, norm

# # Kolmogorov-Smirnov 검정 수행
# statistic, p_value = kstest(data1, 'norm')

# # 검정 통계량과 p-value 출력
# print(f"검정 통계량: {statistic}, p-value: {p_value}")

# # p-value를 기준으로 귀무가설을 기각 여부 판단
# alpha = 0.05
# if p_value > alpha:
#     print("데이터는 정규 분포를 따릅니다 (귀무가설을 기각하지 않음)")
# else:
#     print("데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)")

쇠퇴정상 fillna median

In [118]:
쇠퇴정상[na_col0].fillna(data0.median(), inplace=True)

In [119]:
# 쇠퇴정상[na_col1].fillna(data1.median(), inplace=True)

# 쇠퇴부실

In [120]:
쇠퇴부실 = unlisted_X_train[(unlisted_X_train['쇠퇴기'] == 1) & (unlisted_X_train['부실판단'] == 1)]

결측치 확인

In [121]:
쇠퇴부실.isna().sum().sort_values(ascending=False)

영업재무현금비율             106
회사명                    0
총자본회전률                 0
부가가치(백만원)              0
총자본투자효율                0
설비투자효율                 0
부가가치율                  0
이윤분배율                  0
자본분배율                  0
총자산회전율                 0
매출채권회전률                0
당좌자산회전률                0
유형자산회전율                0
경영자본회전률                0
비유동자산회전률               0
매출액증가율                 0
유동자산회전률                0
영업년수                   0
쭈피처                    0
영업레버리지                 0
재무레버리지                 0
영업투자현금비율               0
도입기                    0
성숙기                    0
성장기                    0
쇠퇴기                    0
유형자산증가율                0
비유동자산증가율               0
거래소코드                  0
현금비율                   0
회계년도                   0
CASH FLOW 대 부채비율       0
CASH FLOW 대 차입금비율      0
CASH FLOW 대 총자본비율      0
CASH FLOW 대 매출액비율      0
유동비율                   0
당좌비율                   0
부채비율                   0
유동부채비율                 0
차입금의존도                 0


정규성 확인

In [122]:
na_col0 = 쇠퇴부실.isna().sum().sort_values(ascending=False).index[0]
# na_col1 = 쇠퇴부실.isna().sum().sort_values(ascending=False).index[1]
# 테스트를 위한 샘플 데이터 생성
data0 = 쇠퇴부실[na_col0].dropna()
# data1 = 쇠퇴부실[na_col1].dropna()

In [123]:
from scipy.stats import shapiro

# 샤피로-윌크 검정 수행
statistic, p_value = shapiro(data0)

# 검정 통계량과 p-value 출력
print(f"검정 통계량: {statistic}, p-value: {p_value}")

# p-value를 기준으로 귀무가설을 기각 여부 판단
alpha = 0.05
if p_value > alpha:
    print("데이터는 정규 분포를 따릅니다 (귀무가설을 기각하지 않음)")
else:
    print("데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)")

검정 통계량: 0.01935398578643799, p-value: 0.0
데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)


In [125]:
# from scipy.stats import shapiro

# # 샤피로-윌크 검정 수행
# statistic, p_value = shapiro(data1)

# # 검정 통계량과 p-value 출력
# print(f"검정 통계량: {statistic}, p-value: {p_value}")

# # p-value를 기준으로 귀무가설을 기각 여부 판단
# alpha = 0.05
# if p_value > alpha:
#     print("데이터는 정규 분포를 따릅니다 (귀무가설을 기각하지 않음)")
# else:
#     print("데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)")

In [126]:
from scipy.stats import anderson

# 안더슨-달링 검정 수행
result = anderson(data0)

# 검정 통계량과 임계값 출력
print(f"검정 통계량: {result.statistic}")

# 임계값과 유의수준에 따른 결과 출력
for i in range(len(result.critical_values)):
    sl, cv = result.significance_level[i], result.critical_values[i]
    print(f"유의수준 {sl}%: 임계값 = {cv}")

# 검정 통계량이 임계값보다 작으면 정규 분포를 따른다고 판단
if result.statistic < result.critical_values[2]:
    print("데이터는 정규 분포를 따릅니다.")
else:
    print("데이터는 정규 분포를 따르지 않습니다.")

검정 통계량: 666.3560402047547
유의수준 15.0%: 임계값 = 0.575
유의수준 10.0%: 임계값 = 0.655
유의수준 5.0%: 임계값 = 0.785
유의수준 2.5%: 임계값 = 0.916
유의수준 1.0%: 임계값 = 1.09
데이터는 정규 분포를 따르지 않습니다.


In [127]:
# from scipy.stats import anderson

# # 안더슨-달링 검정 수행
# result = anderson(data1)

# # 검정 통계량과 임계값 출력
# print(f"검정 통계량: {result.statistic}")

# # 임계값과 유의수준에 따른 결과 출력
# for i in range(len(result.critical_values)):
#     sl, cv = result.significance_level[i], result.critical_values[i]
#     print(f"유의수준 {sl}%: 임계값 = {cv}")

# # 검정 통계량이 임계값보다 작으면 정규 분포를 따른다고 판단
# if result.statistic < result.critical_values[2]:
#     print("데이터는 정규 분포를 따릅니다.")
# else:
#     print("데이터는 정규 분포를 따르지 않습니다.")

In [128]:
from scipy.stats import kstest, norm

# Kolmogorov-Smirnov 검정 수행
statistic, p_value = kstest(data0, 'norm')

# 검정 통계량과 p-value 출력
print(f"검정 통계량: {statistic}, p-value: {p_value}")

# p-value를 기준으로 귀무가설을 기각 여부 판단
alpha = 0.05
if p_value > alpha:
    print("데이터는 정규 분포를 따릅니다 (귀무가설을 기각하지 않음)")
else:
    print("데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)")

검정 통계량: 0.19980988647451559, p-value: 4.839024161896977e-63
데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)


In [129]:
# from scipy.stats import kstest, norm

# # Kolmogorov-Smirnov 검정 수행
# statistic, p_value = kstest(data1, 'norm')

# # 검정 통계량과 p-value 출력
# print(f"검정 통계량: {statistic}, p-value: {p_value}")

# # p-value를 기준으로 귀무가설을 기각 여부 판단
# alpha = 0.05
# if p_value > alpha:
#     print("데이터는 정규 분포를 따릅니다 (귀무가설을 기각하지 않음)")
# else:
#     print("데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)")

median

In [130]:
쇠퇴부실[na_col0].fillna(data0.median(), inplace=True)

In [131]:
# 쇠퇴부실[na_col1].fillna(data1.median(), inplace=True)

# to_csv

train set concat

In [132]:
filled_train_data = pd.concat([도입부실, 도입정상, 성장부실, 성장정상, 성숙부실, 성숙정상, 쇠퇴부실, 쇠퇴정상], axis=0, ignore_index=True)

In [133]:
filled_train_data.columns

Index(['회사명', '거래소코드', '회계년도', 'CASH FLOW 대 부채비율', 'CASH FLOW 대 차입금비율',
       'CASH FLOW 대 총자본비율', 'CASH FLOW 대 매출액비율', '유동비율', '당좌비율', '부채비율',
       '유동부채비율', '차입금의존도', '순운전자본비율', '현금비율', '유동자산구성비율', '자기자본구성비율',
       '자기자본순이익률', '경영자본순이익률', '매출액순이익률', '총자본사업이익률', '총자본영업이익률', '매출액영업이익률',
       '금융비용부담률', 'CROE', '총자산증가율', '총자본증가율', '유동자산증가율', '비유동자산증가율', '매출액증가율',
       '유형자산증가율', '부가가치(백만원)', '총자본투자효율', '설비투자효율', '부가가치율', '이윤분배율', '자본분배율',
       '총자산회전율', '매출채권회전률', '당좌자산회전률', '유형자산회전율', '경영자본회전률', '총자본회전률',
       '비유동자산회전률', '유동자산회전률', '영업년수', '쭈피처', '영업레버리지', '재무레버리지', '영업투자현금비율',
       '영업재무현금비율', '도입기', '성숙기', '성장기', '쇠퇴기', '부실판단'],
      dtype='object')

In [134]:
len(filled_train_data)

15138

In [140]:
filled_train_data.loc[filled_train_data['부실판단'] == 1][['영업레버리지', '재무레버리지']].describe()

,영업레버리지,재무레버리지
count,5470.000,5470.000
mean,8.134,-0.209
std,374.008,47.472
min,-9823.523,-3383.817
25%,-1.548,0.017
50%,0.138,0.683
75%,3.384,1.141
max,24194.818,214.429


In [141]:
filled_train_data.loc[filled_train_data['부실판단'] == 0][['영업레버리지', '재무레버리지']].describe()

,영업레버리지,재무레버리지
count,9668.000,9668.000
mean,10.411,1.520
std,359.302,18.154
min,-6935.789,-983.102
25%,1.491,0.736
50%,2.491,1.020
75%,4.567,1.427
max,31473.399,743.445


In [ ]:
filled_train_data.to_csv('../data/datasets/unlisted_filled_train_data.csv', index=False)

test set

In [ ]:
unlisted_X_test['부실판단'] = unlisted_y_test
unlisted_X_test['부실판단']

19945   0.000
6034    1.000
12336   0.000
19337   0.000
3974    0.000
         ... 
20311   1.000
9086    0.000
9694    0.000
10691   1.000
13411   1.000
Name: 부실판단, Length: 5569, dtype: float64

In [ ]:
unlisted_X_test.to_csv('../data/datasets/unlisted_test_data.csv', index=False)

In [ ]:
unlisted_X_test

,회사명,거래소코드,회계년도,CASH FLOW 대 부채비율,CASH FLOW 대 차입금비율,CASH FLOW 대 총자본비율,CASH FLOW 대 매출액비율,유동비율,당좌비율,부채비율,...,쭈피처,영업레버리지,재무레버리지,영업투자현금비율,영업재무현금비율,도입기,성숙기,성장기,쇠퇴기,부실판단
19945,인지솔라주식회사,87600,2019,15.910,16.110,10.380,69.180,20.290,7.920,187.760,...,0.161,1.871,12.022,-0.793,2.408,0,0,1,0,0.000
6034,(주)케이엔에프인터내셔날,72390,2020,-23.920,-46.320,-21.590,-12.250,105.430,75.350,926.090,...,0.412,-1.397,0.972,-0.777,3.136,0,0,0,1,1.000
12336,경동산업주식회사,10708,2022,6.820,14.720,2.680,1.690,250.570,104.230,64.650,...,0.129,2.443,2.030,-0.880,5.588,0,0,1,0,0.000
19337,(주)장충동왕족발,53614,2022,7.550,10.030,4.640,5.440,64.220,45.060,159.340,...,0.346,11.815,2.035,-1.601,3.127,0,0,1,0,0.000
3974,(주)한강티에스,94815,2022,-8.520,0.000,-6.450,-1.240,171.520,171.520,312.100,...,1.009,-116.667,2.217,-1.019,46.686,0,0,0,1,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20311,조일산업(주),70253,2019,-11.470,-16.560,-7.770,-10.310,83.470,82.450,210.020,...,0.120,-86.965,-0.062,-4.403,-1.000,0,0,0,1,1.000
9086,디앤에이중공업(주),56937,2022,64.790,0.000,14.160,11.660,376.750,293.070,27.960,...,0.140,1.755,0.906,-16.095,-1.849,0,1,0,0,0.000
9694,(주)분더바움,81601,2020,910.550,9831.840,304.530,55.910,268.100,240.150,50.250,...,0.326,5.194,1.226,-7.998,-1.144,0,1,0,0,0.000
10691,빅스포주식회사,50162,2022,-3.970,-4.310,-2.550,-9.910,29.760,26.890,178.580,...,0.426,-1.137,0.781,-0.933,0.971,0,0,0,1,1.000
